In [1]:
!pip install --upgrade --user hmmlearn
from hmmlearn import hmm
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import confusion_matrix
from scipy.stats import multivariate_normal
import math

Requirement already up-to-date: hmmlearn in /root/.local/lib/python3.6/site-packages (0.2.3)


In [0]:
#EDIT DATASETS HERE 
#USE DATASET VARIABLE FOR FINAL DATASET
dataset=pd.read_csv('Monday-WorkingHours.pcap_ISCX.csv')
#dataset=pd.read_csv('Tuesday-WorkingHours.pcap_ISCX.csv')
dataset_1=pd.read_csv('Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')
#dataset=pd.read_csv('Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv')
#dataset=pd.read_csv('Wednesday-workingHours.pcap_ISCX.csv')
#dataset=pd.read_csv('Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')

In [3]:
#CONCATENATE DATASETS
dataset=dataset.append(dataset_1, ignore_index= True)
#dataset=dataset.append(dataset_2, ignore_index= True)
#dataset=dataset.append(dataset_3, dataset_4,ignore_index=True)
#dataset=dataset.append(dataset_4, ignore_index=True)
dataset

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.000000,0.000000,0,0,0.000000,0.000000,3.000000e+06,5.000000e+05,4.000000,0.000000,4,4,4,4.000000,0.000000,4,4,0,0.000000,0.000000,0,0,0,0,0,0,40,0,5.000000e+05,0.000000,6,6,6.000000,0.000000,0.000,0,0,0,0,1,1,0,0,0,9.000000,6.000000,0.000000,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.000000,0.000000,0,0,0.000000,0.000000,1.200000e+07,2.000000e+06,1.000000,0.000000,1,1,1,1.000000,0.000000,1,1,0,0.000000,0.000000,0,0,0,0,0,0,40,0,2.000000e+06,0.000000,6,6,6.000000,0.000000,0.000,0,0,0,0,1,1,0,0,0,9.000000,6.000000,0.000000,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.000000,0.000000,0,0,0.000000,0.000000,1.200000e+07,2.000000e+06,1.000000,0.000000,1,1,1,1.000000,0.000000,1,1,0,0.000000,0.000000,0,0,0,0,0,0,40,0,2.000000e+06,0.000000,6,6,6.000000,0.000000,0.000,0,0,0,0,1,1,0,0,0,9.000000,6.000000,0.000000,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,49188,1,2,0,12,0,6,6,6.000000,0.000000,0,0,0.000000,0.000000,1.200000e+07,2.000000e+06,1.000000,0.000000,1,1,1,1.000000,0.000000,1,1,0,0.000000,0.000000,0,0,0,0,0,0,40,0,2.000000e+06,0.000000,6,6,6.000000,0.000000,0.000,0,0,0,0,1,1,0,0,0,9.000000,6.000000,0.000000,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,49486,3,2,0,12,0,6,6,6.000000,0.000000,0,0,0.000000,0.000000,4.000000e+06,6.666667e+05,3.000000,0.000000,3,3,3,3.000000,0.000000,3,3,0,0.000000,0.000000,0,0,0,0,0,0,40,0,6.666667e+05,0.000000,6,6,6.000000,0.000000,0.000,0,0,0,0,1,1,0,0,0,9.000000,6.000000,0.000000,40,0,0,0,0,0,0,2,12,0,0,245,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816380,443,196135,49,57,1331,105841,570,0,27.163265,108.067176,4344,0,1856.859649,972.796621,5.464196e+05,5.404441e+02,1867.952381,7207.469062,53438,1,196135,4086.145833,10366.095180,53521,1,172901,3087.517857,9307.416597,53438,2,0,0,0,0,1252,1832,2.498279e+02,290.616157,0,4344,1001.607477,1160.591064,1346971.618,0,0,0,1,0,0,0,0,1,1011.056604,27.163265,1856.859649,1252,0,0,0,0,0,0,49,1331,57,105841,29200,61,31,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
816381,443,378424,49,59,1325,104393,570,0,27.040816,108.095051,2896,0,1769.372881,848.453540,2.793639e+05,2.853942e+02,3536.672897,18326.251010,173388,0,378424,7883.833333,34491.462460,236489,2,355402,6127.620690,27253.356700,196547,1,0,0,0,0,1264,189

In [4]:
n_features=40
dataset.head()
rows,cols=dataset.shape
rows, cols

(816385, 79)

In [0]:
#removing NaN's and np.inf from the dataset
#df[df==np.inf]=np.nan 
#df.fillna(df.mean(), inplace=True)

In [0]:
dataset[dataset==np.inf]=np.nan
dataset.fillna(0,inplace=True)

In [0]:
x_dataset=dataset.iloc[:, 0:n_features]
y=dataset.iloc[:, cols-1]

In [8]:
le=LabelEncoder()
y_encoded=le.fit_transform(y)
le.classes_

array(['BENIGN', 'PortScan'], dtype=object)

In [9]:
classes=le.classes_
x=[0 for j in range(0,len(classes))]
for i in range(0, len(classes)):
  class_string=classes[i]
  x[i]=dataset[dataset.iloc[:,cols-1]== class_string] 
x=[x[i].iloc[:,0:n_features] for i in range(0,len(classes))]
x[0].shape

(657455, 40)

In [0]:
scaler=MinMaxScaler()
x_dataset_scaled=scaler.fit_transform(x_dataset)
for i in range(0,len(classes)):
  x[i]=scaler.fit_transform(x[i])

In [0]:
x_train,x_test_real,y_train,y_test=train_test_split(x_dataset_scaled, y_encoded, test_size=0.3)

In [12]:
n_states=3
T=10
n_attacks=len(classes)
n_attacks

2

In [0]:
model=[hmm.GaussianHMM(n_states) for i in range(0,n_attacks)]
for i in range(0,n_attacks):
  model[i].fit(x[i])

In [0]:
def forward(B,A,pi):
  alpha = [[[0 for i in range(n_states)] for j in range(len(x_test))] for k in range(0,n_attacks)] 
  for k in range(0,n_attacks):
    for i in range(0,n_states):
      alpha[k][0][i]=pi[k][i]*B[k][0][i]
  sum=0
  for k in range(0,n_attacks):
    for t in range(0,len(x_test)-1):
      for j in range (0, n_states):
        for i in range(0,n_states):
          sum=sum+alpha[k][t][i]*A[k][i][j]
        sum=sum*B[k][t+1][j]
        alpha[k][t+1][j]=sum
      sum=0 #i think there is one indent extra here. please recheck.
  prob_obs=[0 for k in range(0,n_attacks)]
  prob=0
  for k in range(0,n_attacks):
    for i in range(0,n_states):
      prob=prob+alpha[k][len(x_test)-1][i]
    prob_obs[k]=prob
    prob=0
  return prob_obs

In [15]:
index=0
n_times=int(len(x_test_real)/T)
y_pred=[1 for i in range(0,len(x_test_real))]
for T_obs in range(0,n_times):
  x_test=x_test_real[index:index+T,:]
  B=[model[i].predict_proba(x_test) for i in range(0,n_attacks)]
#B=[model[i]._compute_log_likelihood(x_test) for i in range(0,n_attacks)]
  A=[model[i].transmat_ for i in range(0,n_attacks)]
  pi=[model[i].startprob_ for i in range(0,n_attacks)]
  prob_obs=forward(B,A,pi)
  #print(prob_obs)
#ASSIGN LABELS
  pred_label=prob_obs.index(max(prob_obs))
  for i in range(index, index+T):
    y_pred[i]=pred_label
  index=index+T
len(y_pred)

244916

In [16]:
print(confusion_matrix(y_test,y_pred))
accuracy_score(y_test,y_pred)

[[192038   5085]
 [ 47082    711]]


0.7870004409675154